# Regression Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

I {**YOUR NAME, YOUR SURNAME**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Predict Overview: Spain Electricity Shortfall Challenge

The government of Spain is considering an expansion of it's renewable energy resource infrastructure investments. As such, they require information on the trends and patterns of the countries renewable sources and fossil fuel energy generation. Your company has been awarded the contract to:

- 1. analyse the supplied data;
- 2. identify potential errors in the data and clean the existing data set;
- 3. determine if additional features can be added to enrich the data set;
- 4. build a model that is capable of forecasting the three hourly demand shortfalls;
- 5. evaluate the accuracy of the best machine learning model;
- 6. determine what features were most important in the model’s prediction decision, and
- 7. explain the inner working of the model to a non-technical audience.

Formally the problem statement was given to you, the senior data scientist, by your manager via email reads as follow:

> In this project you are tasked to model the shortfall between the energy generated by means of fossil fuels and various renewable sources - for the country of Spain. The daily shortfall, which will be referred to as the target variable, will be modelled as a function of various city-specific weather features such as `pressure`, `wind speed`, `humidity`, etc. As with all data science projects, the provided features are rarely adequate predictors of the target variable. As such, you are required to perform feature engineering to ensure that you will be able to accurately model Spain's three hourly shortfalls.
 
On top of this, she has provided you with a starter notebook containing vague explanations of what the main outcomes are. 

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Importing Packages ⚡ |
| :--------------------------- |
| In this section you are required to import, and briefly discuss, the libraries that will be used throughout your analysis and modelling. |

---
`Add explanation for inclusion of each package and description of its function`

In [ ]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Libraries for data preparation and model building
import scipy as sp
import statsmodels as sm
import sklearn.model_selection as skl


# Setting global constants to ensure notebook results are reproducible
PARAMETER_CONSTANT = 42

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data ⚡ |
| :--------------------------- |
| In this section you are required to load the data from the `df_train` file into a DataFrame. |

---

In [ ]:
df = pd.read_csv('df_train.csv')

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
 ⚡ Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, you are required to perform an in-depth analysis of all the variables in the DataFrame. |

---

`Add description of findings in a markdown cell.`


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Checks for total nulls in the dataframe
df.isnull().sum().sum()

In [ ]:
df.info()

In [ ]:
#  Checks for total per column
def check_nulls (data):
    null_dict = {}

    for col in df.columns:

        if df[col].size > df[col].count():
            nulls = df[col].size - df[col].count()
            percent_missing = round(nulls / df[col].size * 100)
            null_dict[col] = f'has {nulls} nulls making up {percent_missing}% missing values'
    return null_dict

In [ ]:
check_nulls(df)

In [ ]:
# Dropping the column "Unnamed: 0" because it is a repeat of the row index. 
df = df.drop("Unnamed: 0", axis =1)

In [31]:
# 1. Order the columns in alphabetical order
df = df.reindex(sorted(df.columns), axis=1)

# 2. Keep the "time" column in the first index position
df = df[['time'] + [col for col in df.columns if col != 'time']]

# 3. Keep the "load_shortfall_3h" column last
df = df[[col for col in df.columns if col != 'load_shortfall_3h'] + ['load_shortfall_3h']]

# 4. Convert all column titles to lowercase
df.columns = df.columns.str.lower()

df.head()

,time,barcelona_pressure,barcelona_rain_1h,barcelona_rain_3h,barcelona_temp,barcelona_temp_max,barcelona_temp_min,barcelona_weather_id,barcelona_wind_deg,barcelona_wind_speed,...,seville_wind_speed,valencia_humidity,valencia_pressure,valencia_snow_3h,valencia_temp,valencia_temp_max,valencia_temp_min,valencia_wind_deg,valencia_wind_speed,load_shortfall_3h
0,2015-01-01 03:00:00,1036.333333,0.0,0.0,281.013000,281.013000,281.013000,800.0,42.666667,6.333333,...,3.333333,75.666667,1002.666667,0.0,269.888000,269.888000,269.888000,level_5,0.666667,6715.666667
1,2015-01-01 06:00:00,1037.333333,0.0,0.0,280.561667,280.561667,280.561667,800.0,139.000000,4.000000,...,3.333333,71.000000,1004.333333,0.0,271.728333,271.728333,271.728333,level_10,1.666667,4171.666667
2,2015-01-01 09:00:00,1038.000000,0.0,0.0,281.583667,281.583667,281.583667,800.0,326.000000,2.000000,...,2.666667,65.666667,1005.333333,0.0,278.008667,278.008667,278.008667,level_9,1.000000,4274.666667
3,2015-01-01 12:00:00,1037.000000,0.0,0.0,283.434104,283.434104,283.434104,800.0,273.000000,2.333333,...,4.000000,54.000000,1009.000000,0.0,284.899552,284.899552,284.899552,level_8,1.000000,5075.666667
4,2015-01-01 15:00:00,1035.000000,0.0,0.0,284.213167,284.213167,284.213167,800.0,260.000000,4.333333,...,3.000000,58.333333,NaN,0.0,283.015115,283.015115,283.015115,level_7,1.000000,6620.666667


In [34]:
#checking number of unique entries per column
df.nunique()

time                    8763
barcelona_pressure       189
barcelona_rain_1h         31
barcelona_rain_3h         77
barcelona_temp          4233
barcelona_temp_max      2648
barcelona_temp_min      2572
barcelona_weather_id     269
barcelona_wind_deg       974
barcelona_wind_speed      39
bilbao_clouds_all        277
bilbao_pressure          205
bilbao_rain_1h            20
bilbao_snow_3h            83
bilbao_temp             6676
bilbao_temp_max         3700
bilbao_temp_min         3542
bilbao_weather_id        356
bilbao_wind_deg         1042
bilbao_wind_speed         39
madrid_clouds_all        250
madrid_humidity          280
madrid_pressure          298
madrid_rain_1h            19
madrid_temp             6810
madrid_temp_max         3606
madrid_temp_min         3856
madrid_weather_id        289
madrid_wind_speed         38
seville_clouds_all       246
seville_humidity         274
seville_pressure          25
seville_rain_1h           17
seville_rain_3h           50
seville_temp  

In [ ]:
# plot relevant feature interactions

In [ ]:
# evaluate correlation

In [ ]:
# have a look at feature distributions

<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section you are required to: clean the dataset, and possibly create new features - as identified in the EDA phase. |

---

In [ ]:
# remove missing values/ features

In [ ]:
# create new features

In [ ]:
# engineer existing features

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, you are required to create one or more regression models that are able to accurately predict the thee hour load shortfall. |

---

In [ ]:
# split data

In [ ]:
# create targets and features dataset

In [ ]:
# create one or more ML models

In [ ]:
# evaluate one or more ML models

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section you are required to compare the relative performance of the various trained ML models on a holdout dataset and comment on what model is the best and why. |

---

In [ ]:
# Compare model performance

In [ ]:
# Choose best model and motivate why it is the best choice

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, you are required to discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

In [ ]:
# discuss chosen methods logic